## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import random
import requests
import os
import matplotlib.pyplot as plt
import ipywidgets as widgets
import gmaps.datasets
import gmaps.geojson_geometries
import ipywidgets as widgets
widgets.IntSlider()
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head(5)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Description
0,0,Eyl,7.9803,49.8164,76.77,83,14,26.40,SO,few clouds
1,1,Nikolskoye,59.7035,30.7861,67.89,94,100,4.47,RU,overcast clouds
2,2,Busselton,-33.6500,115.3333,55.09,78,100,24.29,AU,overcast clouds
3,3,Souillac,-20.5167,57.5167,69.37,100,75,3.44,MU,moderate rain
4,4,Rikitea,-23.1203,-134.9692,74.19,75,2,11.81,PF,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Description
0,0,Eyl,7.9803,49.8164,76.77,83,14,26.40,SO,few clouds
1,1,Nikolskoye,59.7035,30.7861,67.89,94,100,4.47,RU,overcast clouds
2,2,Busselton,-33.6500,115.3333,55.09,78,100,24.29,AU,overcast clouds
3,3,Souillac,-20.5167,57.5167,69.37,100,75,3.44,MU,moderate rain
4,4,Rikitea,-23.1203,-134.9692,74.19,75,2,11.81,PF,clear sky
7,7,Australind,-33.2667,115.7167,55.06,82,100,21.74,AU,overcast clouds
9,9,Puerto Ayora,-0.7393,-90.3518,75.16,85,64,8.99,EC,broken clouds
10,10,Port Hedland,-20.3167,118.5667,57.25,77,6,6.91,AU,clear sky
11,11,Zalantun,48.0000,122.7167,66.20,99,100,3.60,CN,light rain
13,13,Esperance,-33.8667,121.9000,58.84,80,3,14.20,AU,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID        405
City           405
Lat            405
Lng            405
Max Temp       405
Humidity       405
Cloudiness     405
Wind Speed     405
Country        401
Description    405
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
preferred_cities_df.count()

City_ID        401
City           401
Lat            401
Lng            401
Max Temp       401
Humidity       401
Cloudiness     401
Wind Speed     401
Country        401
Description    401
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Eyl,SO,76.77,few clouds,7.9803,49.8164,
1,Nikolskoye,RU,67.89,overcast clouds,59.7035,30.7861,
2,Busselton,AU,55.09,overcast clouds,-33.6500,115.3333,
3,Souillac,MU,69.37,moderate rain,-20.5167,57.5167,
4,Rikitea,PF,74.19,clear sky,-23.1203,-134.9692,
7,Australind,AU,55.06,overcast clouds,-33.2667,115.7167,
9,Puerto Ayora,EC,75.16,broken clouds,-0.7393,-90.3518,
10,Port Hedland,AU,57.25,clear sky,-20.3167,118.5667,
11,Zalantun,CN,66.20,light rain,48.0000,122.7167,
13,Esperance,AU,58.84,clear sky,-33.8667,121.9000,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
   
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json" 

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Not found. Skipping")
    pass

Not found. Skipping
Not found. Skipping
Not found. Skipping
Not found. Skipping
Not found. Skipping
Not found. Skipping
Not found. Skipping
Not found. Skipping
Not found. Skipping
Not found. Skipping
Not found. Skipping
Not found. Skipping
Not found. Skipping
Not found. Skipping
Not found. Skipping
Not found. Skipping
Not found. Skipping
Not found. Skipping
Not found. Skipping
Not found. Skipping
Not found. Skipping
Not found. Skipping
Not found. Skipping
Not found. Skipping
Not found. Skipping
Not found. Skipping
Not found. Skipping
Not found. Skipping
Not found. Skipping
Not found. Skipping
Not found. Skipping
Not found. Skipping
Not found. Skipping
Not found. Skipping
Not found. Skipping
Not found. Skipping


In [9]:
# 7. Drop the rows where there is no Hotel Name.
all_hotel_df = hotel_df.drop(hotel_df[hotel_df["Hotel Name"] == ""].index)
all_hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Eyl,SO,76.77,few clouds,7.9803,49.8164,Amiin Hotel
1,Nikolskoye,RU,67.89,overcast clouds,59.7035,30.7861,Tourist House - Sablino
2,Busselton,AU,55.09,overcast clouds,-33.6500,115.3333,Observatory Guest House
3,Souillac,MU,69.37,moderate rain,-20.5167,57.5167,Shanti Maurice Resort & Spa
4,Rikitea,PF,74.19,clear sky,-23.1203,-134.9692,People ThankYou
7,Australind,AU,55.06,overcast clouds,-33.2667,115.7167,ON THE ESTUARY
9,Puerto Ayora,EC,75.16,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
10,Port Hedland,AU,57.25,clear sky,-20.3167,118.5667,The Esplanade Hotel
11,Zalantun,CN,66.20,light rain,48.0000,122.7167,GreenTree Alliance Hulunbeier Zhalantun Railwa...
13,Esperance,AU,58.84,clear sky,-33.8667,121.9000,Hospitality Esperance


In [10]:
# validating we don't have empty fields
all_hotel_df.count()

City           365
Country        365
Max Temp       365
Description    365
Lat            365
Lng            365
Hotel Name     365
dtype: int64

In [11]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
all_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current weather</dt><dd>{Description}and{Max Temp} ºF</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in all_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = all_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
locations = all_hotel_df[["Lat", "Lng"]]
max_temp = all_hotel_df["Max Temp"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level = 1.5)

heat_layer = gmaps.heatmap_layer(locations, weights = max_temp, dissipating = False, max_intensity = 300, point_radius = 4)
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
# fig = gmaps.figure()
fig

Figure(layout=FigureLayout(height='420px'))